In [ ]:
import sys
import glob
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.interpolate import interp1d
import statsmodels.api as sm

import matplotlib.pyplot as plt
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor, tailcuts_clean
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from astropy.table import Table, vstack

#from muonpipe import usefull_func
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import copy


In [ ]:
filename = '/Users/vdk/muons2024//simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

## `mirror_align_random` = 0.015 deg

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment

## `mirror_align_random` = 0.0025, 0.0046, 0.0092, 0.02

In [ ]:
import glob
from astropy.table import Table, vstack
import pandas as pd

# Define a function to read and process FITS files
def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

# Define a function to filter the DataFrame
def filter_dataframe(df, muon_efficiency_cut, size_outside_cut):
    return df[
        df['good_ring'] & 
        (df['muon_efficiency'] < muon_efficiency_cut) & 
        (df['size_outside'] < size_outside_cut)
    ]

# Process the first set of files
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_00025alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00025_alignment = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

# Process the second set of files
listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00043_alignment = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

# Process the third set of files
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_00092alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_0092_alignment = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

# Process the fourth set of files
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_002alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_002_alignment = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


# Data load

In [ ]:
muon_files = glob.glob('/Users/vdk/Documents/DocumentsVadymMacBookWork/all_muon_fits/low_nsb_filter/*')
for i, filename in enumerate(sorted(muon_files)):
    print(i, filename)

In [ ]:
df_files = [
    pd.read_csv(muon_file, na_values=['NA', '?']).query(
        '(muon_efficiency < @muon_efficiency_cut) & '
        '(size_outside < @size_outside_cut) & '
        '(event_time >= @start_date_2024) & '
        '(event_time <= @end_date_2024)'
    )
    for muon_file in sorted(muon_files)[9:]
]

df_good_data_real = pd.concat(df_files, ignore_index=True)
df_good_data_real

In [ ]:
# df_good_data_real['event_time'] = pd.to_datetime(df_good_data_real['event_time'], unit='s')
# df_good_data_real['event_time'] = df_good_data_real['event_time'].dt.strftime('%d%m%Y')
# min(df_good_data_real['event_time'])

max(df_good_data_real['event_time'])

# Cuts

In [ ]:
ring_completeness_cut = 0.99
ring_containment_cut = 0.99
min_radius_cut = 1
max_radius_cut = 1.16
ring_center_distance_cut = 0.6
impact_distance_cut = 6

# ring_completeness_cut = 0.4
# ring_containment_cut = 0
# min_radius_cut = 0.6
# max_radius_cut = 1.4
# ring_center_distance_cut = 2
# impact_distance_cut = 15


In [ ]:
def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_x'].abs() < center_distance_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut)
    ]


# Apply cuts to each dataframe
df_data_cut = apply_cuts(df_good_data_real, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%) out of {len(df_good_data_real)}")

df_sim_00025_cut = apply_cuts(df_sim_00025_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00025_cut: {len(df_sim_00025_cut)} ({calculate_survival_percentage(df_sim_00025_alignment, df_sim_00025_cut):.2f}%) out of {len(df_sim_00025_alignment)}")

df_sim_00043_cut = apply_cuts(df_sim_00043_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00043_cut)} ({calculate_survival_percentage(df_sim_00043_alignment, df_sim_00043_cut):.2f}%) out of {len(df_sim_00043_alignment)}")

df_sim_00092_cut = apply_cuts(df_sim_0092_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00092_cut: {len(df_sim_00092_cut)} ({calculate_survival_percentage(df_sim_0092_alignment, df_sim_00092_cut):.2f}%) out of {len(df_sim_0092_alignment)}")

df_sim_0015_cut = apply_cuts(df_sim_0015_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_cut):.2f}%) out of {len(df_sim_0015_alignment)}")

df_sim_002_cut = apply_cuts(df_sim_002_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_002_cut: {len(df_sim_002_cut)} ({calculate_survival_percentage(df_sim_002_alignment, df_sim_002_cut):.2f}%) out of {len(df_sim_002_alignment)}")


In [ ]:
bin_number = 100
plt.figure(figsize=(14, 10))
plt.xlim(0,0.2)
#plt.ylim(0,1)
z,x,c = plt.hist(df_data_cut['ring_width'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3, color = 'k')
z,x,c = plt.hist(df_sim_0015_cut['ring_width'], bins=bin_number, alpha=0.8, label='Sim+additional cuts+mirror_align_random=0.015deg', density=True, histtype='step', linewidth=3, color = palette[1])

mean_data = df_data_cut['ring_width'].mean()
std_data = df_data_cut['ring_width'].std()
mean_0015 = df_sim_0015_cut['ring_width'].mean()
std_0015 = df_sim_0015_cut['ring_width'].std()

# Print mean and std on the figure
plt.text(0.12, 16, f"Mean: {mean_data:.3f}, Std: {std_data:.3f}", fontsize=20, color='k')
plt.text(0.12, 15, f"Mean: {mean_0015:.3f}, Std: {std_0015:.3f}", fontsize=20, color=palette[1])

plt.legend(prop={'size': 15})
plt.xlabel('Ring Width [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.tick_params(axis='both', labelsize=15)
plt.tight_layout()
#plt.savefig(f'/Users/vdk/muons2024/psf_work/lst_general_meeting/2024year_width_hist_completeness>{ring_completeness_cut}_radius_between{min_radius_cut,max_radius_cut}_abs(center)<{ring_center_distance_cut}_impact_cut{impact_distance_cut}.pdf', dpi=200)

In [ ]:
bin_number = 10
plt.figure(figsize=(12, 10))
plt.xlim(0.95,1.2)
z,x,c = plt.hist(df_data_cut['ring_radius'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3, color= 'k')
z,x,c = plt.hist(df_sim_0015_cut['ring_radius'], bins=bin_number, alpha=0.8, label='Sim+additional cuts+mirror_align_random=0.015deg', density=True, histtype='step', linewidth=3, color=palette[1])
#z,x,c = plt.hist(df_sim_001_alignment['ring_radius'], bins=bin_number, alpha=0.8, label='0.01 deg', density=True, histtype='step', linewidth=3)
#z,x,c = plt.hist(df_sim_eslope['ring_radius'], bins=bin_number, alpha=0.8, label='eslope', density=True, histtype='step', linewidth=3)

plt.legend(prop={'size': 15})
plt.xlabel('Ring Radius [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.tick_params(axis='both', labelsize=15)
plt.tight_layout()
#plt.savefig(f'/Users/vdk/muons2024/psf_work/lst_general_meeting/2024year_radius_hist_completeness>{ring_completeness_cut}_radius_between{min_radius_cut,max_radius_cut}_abs(center)<{ring_center_distance_cut}_impact_cut{impact_distance_cut}.pdf', dpi=200)

# Creating binned distribution

In [ ]:
# Define the number of entries to use from each dataset
MAX_ENTRIES = 2100000
NUM_BINS = 10
COMPLETENESS_CUT = ring_completeness_cut
QUANTILE_CUT = 0.2

# Calculate the 20th percentile (quantile 0.2) for each dataset
quantiles = {
    'data': df_data_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_0015_alignment': df_sim_0015_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_00043_alignment': df_sim_00043_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_00025_alignment': df_sim_00025_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_00092_alignment': df_sim_00092_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_002_alignment': df_sim_002_cut['ring_width'].quantile(QUANTILE_CUT)
}

# Define a list of tuples containing each dataframe, its corresponding quantile, and label
full_datasets = [
    (df_data_cut[df_data_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data'),
    (df_sim_0015_cut[df_sim_0015_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
    (df_sim_00043_cut[df_sim_00043_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_00043_alignment'], '0.0043 deg'),
    (df_sim_00025_cut[df_sim_00025_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_00025_alignment'], '0.0025 deg'),
    (df_sim_00092_cut[df_sim_00092_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_00092_alignment'], '0.0092 deg'),
    (df_sim_002_cut[df_sim_002_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_002_alignment'], '0.02 deg')
]

# Assuming 'datasets' is a list of tuples: (df, quantile, label)
# and MAX_ENTRIES and NUM_BINS are predefined constants.

# First, process all datasets to filter and collect ring_radius values
filtered_dfs = []

for df, quantile, label in full_datasets:
    # Limit to the first MAX_ENTRIES entries
    limited_df = df.iloc[:MAX_ENTRIES]
    
    # Filter the data based on the quantile
    filtered_df = limited_df[limited_df['ring_width'] < quantile].copy()
    
    # Check if filtered_df is not empty to avoid errors
    if filtered_df.empty:
        print(f"Warning: No data points below the 20th percentile for '{label}' in the first {MAX_ENTRIES} entries.")
        continue
    
    filtered_df['label'] = label  # Keep track of the label
    filtered_dfs.append(filtered_df)

# Concatenate all filtered DataFrames to determine common bin edges
combined_filtered_df = pd.concat(filtered_dfs, ignore_index=True)

# Determine the common bin edges based on the combined data
min_radius = combined_filtered_df['ring_radius'].min()
max_radius = combined_filtered_df['ring_radius'].max()

# Create common bin edges
bin_edges = np.linspace(min_radius, max_radius, NUM_BINS + 1)

# Compute bin centers
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Map bin index to bin center
bin_index_to_center = dict(enumerate(bin_centers))

# Initialize dictionary to store binned data
binned_data = {}

for filtered_df in filtered_dfs:
    label = filtered_df['label'].iloc[0]  # Retrieve the label
    
    # Bin 'ring_radius' using the common bin_edges
    filtered_df['radius_bin'] = pd.cut(
        filtered_df['ring_radius'], 
        bins=bin_edges, 
        labels=False, 
        include_lowest=True
    )
    
    # Remove any rows with NaN in 'radius_bin'
    filtered_df = filtered_df.dropna(subset=['radius_bin'])
    
    # Convert 'radius_bin' to integer
    filtered_df['radius_bin'] = filtered_df['radius_bin'].astype(int)
    
    # Group by 'radius_bin' and compute required statistics
    grouped = filtered_df.groupby('radius_bin').agg({
        'ring_width': ['mean', 'std', 'count']
    }).reset_index()
    
    # Flatten MultiIndex columns
    grouped.columns = [
        'radius_bin', 
        'ring_width_mean', 
        'ring_width_std', 
        'count'
    ]
    
    # Add bin centers to the grouped DataFrame
    grouped['ring_radius_center'] = grouped['radius_bin'].map(bin_index_to_center)
    
    # Compute width/radius ratio using bin centers
    grouped['width_radius_ratio'] = grouped['ring_width_mean'] / grouped['ring_radius_center']
    
    # Compute statistical error (standard error of the mean) for 'ring_width_mean'
    grouped['ring_width_sem'] = grouped['ring_width_std'] / np.sqrt(grouped['count'])
    
    # Compute error for width/radius ratio (error propagation)
    grouped['width_radius_ratio_sem'] = grouped['width_radius_ratio'] * (
        grouped['ring_width_sem'] / grouped['ring_width_mean']
    )
    
    # Store the grouped data
    binned_data[label] = grouped

# Plotting

In [ ]:
# Define the number of entries to use from each dataset
MAX_ENTRIES = 2100000
NUM_BINS = 10
COMPLETENESS_CUT = ring_completeness_cut
QUANTILE_CUT = 0.2


datasets = copy.deepcopy(full_datasets[:2])

# First, process all datasets to filter and collect ring_radius values
filtered_dfs = []

for df, quantile, label in datasets:
    # Limit to the first MAX_ENTRIES entries
    limited_df = df.iloc[:MAX_ENTRIES]
    
    # Filter the data based on the quantile
    filtered_df = limited_df[limited_df['ring_width'] < quantile].copy()
    
    # Check if filtered_df is not empty to avoid errors
    if filtered_df.empty:
        print(f"Warning: No data points below the 20th percentile for '{label}' in the first {MAX_ENTRIES} entries.")
        continue
    
    filtered_df['label'] = label  # Keep track of the label
    filtered_dfs.append(filtered_df)

# Concatenate all filtered DataFrames to determine common bin edges
combined_filtered_df = pd.concat(filtered_dfs, ignore_index=True)

# Determine the common bin edges based on the combined data
min_radius = combined_filtered_df['ring_radius'].min()
max_radius = combined_filtered_df['ring_radius'].max()

# Create common bin edges
bin_edges = np.linspace(min_radius, max_radius, NUM_BINS + 1)

# Compute bin centers
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Map bin index to bin center
bin_index_to_center = dict(enumerate(bin_centers))

# Initialize dictionary to store binned data
binned_data = {}

for filtered_df in filtered_dfs:
    label = filtered_df['label'].iloc[0]  # Retrieve the label
    
    # Bin 'ring_radius' using the common bin_edges
    filtered_df['radius_bin'] = pd.cut(
        filtered_df['ring_radius'], 
        bins=bin_edges, 
        labels=False, 
        include_lowest=True
    )
    
    # Remove any rows with NaN in 'radius_bin'
    filtered_df = filtered_df.dropna(subset=['radius_bin'])
    
    # Convert 'radius_bin' to integer
    filtered_df['radius_bin'] = filtered_df['radius_bin'].astype(int)
    
    # Group by 'radius_bin' and compute required statistics
    grouped = filtered_df.groupby('radius_bin').agg({
        'ring_width': ['mean', 'std', 'count']
    }).reset_index()
    
    # Flatten MultiIndex columns
    grouped.columns = [
        'radius_bin', 
        'ring_width_mean', 
        'ring_width_std', 
        'count'
    ]
    
    # Add bin centers to the grouped DataFrame
    grouped['ring_radius_center'] = grouped['radius_bin'].map(bin_index_to_center)
    
    # Compute width/radius ratio using bin centers
    grouped['width_radius_ratio'] = grouped['ring_width_mean'] / grouped['ring_radius_center']
    
    # Compute statistical error (standard error of the mean) for 'ring_width_mean'
    grouped['ring_width_sem'] = grouped['ring_width_std'] / np.sqrt(grouped['count'])
    
    # Compute error for width/radius ratio (error propagation)
    grouped['width_radius_ratio_sem'] = grouped['width_radius_ratio'] * (
        grouped['ring_width_sem'] / grouped['ring_width_mean']
    )
    
    # Store the grouped data
    binned_data[label] = grouped

# Plot settings
alpha_reg = 0.7
palette = ['k'] + sns.color_palette("husl", len(binned_data))[:-1]

plt.figure(figsize=(12, 10))
flag = '2023-24'

# Iterate over each binned dataset and plot with regression lines and confidence intervals
for i, (label, grouped_df) in enumerate(binned_data.items()):
    sns.regplot(
        x='ring_radius_center',
        y='width_radius_ratio',
        data=grouped_df,
        label=label,
        scatter_kws={'alpha': alpha_reg, 's': 100},
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

# Customize plot labels and limits
plt.xlabel('Muon Radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
plt.ylim(0.03, 0.08)
plt.xlim(min_radius_cut, max_radius_cut)

# Enhance the legend
#plt.legend(prop={'size': 26}, loc='lower right', frameon=True)
plt.legend(ncol=2, fontsize=26, loc='upper right', frameon=True)

# Add a title for context
#plt.title(f'Ring width vs radius ratio for 2024 year, {QUANTILE_CUT*100}% quantile, completeness > {COMPLETENESS_CUT}', fontsize=18)

# Improve layout and display the plot
plt.tight_layout()
plt.grid(alpha=0.3)
plt.tick_params(axis='both', labelsize=15)

# Optionally, save the figure
#plt.savefig(f'/Users/vdk/muons2024/psf_work/lst_general_meeting/widthradius_vs_radius_completeness_{COMPLETENESS_CUT}_quantile{QUANTILE_CUT*100}%_radiusrange{min_radius_cut,max_radius_cut}_center{ring_center_distance_cut}_impact{impact_distance_cut}nsb_tune2024.pdf', dpi=200)
plt.show()

In [ ]:
datasets = copy.deepcopy(full_datasets[:1])

# Define your alpha and palette if not already defined
alpha_reg = 0.7  # Adjust as needed
palette = ['k'] + sns.color_palette("husl", len(binned_data))[:-1]

plt.figure(figsize=(10, 8))

for i, (label, grouped_df) in enumerate(binned_data.items()):
    # Plot the data points with error bars
    plt.errorbar(
        x=grouped_df['ring_radius_center'],
        y=grouped_df['width_radius_ratio'],
        yerr=grouped_df['ring_width_sem'] / grouped_df['ring_radius_center'],  # Error propagation for ratio
        fmt='o',
        label=label,
        color=palette[i],
        alpha=alpha_reg,
        capsize=4
    )
    
    # Fit and plot the regression line
    sns.regplot(
        x='ring_radius_center',
        y='width_radius_ratio',
        data=grouped_df,
        scatter=False,  # We already plotted the scatter with error bars
        #label=f'{label} Fit',
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

plt.xlabel('Muon ring radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
#plt.title('Muon Ring Width vs. Ring Radius')
plt.legend(ncol=2, fontsize=26, loc='upper right', frameon=True)

plt.tight_layout()
plt.xlim(1,1.16)
plt.ylim(0.03, 0.07)
plt.grid(alpha=0.4)
plt.tick_params(axis='both', labelsize=15)
#plt.savefig(f'/Users/vdk/muons2024/psf_work/lst_general_meeting/statistica_errors_widthradius_vs_radius_completeness_{COMPLETENESS_CUT}_quantile{QUANTILE_CUT*100}%_radiusrange{min_radius_cut,max_radius_cut}_center{ring_center_distance_cut}_impact{impact_distance_cut}nsb_tune2024.pdf', dpi=200)
plt.show()

## Only for 2 datasets

In [ ]:
datasets = copy.deepcopy(full_datasets[:1])

# Define your alpha and palette if not already defined
alpha_reg = 0.7  # Adjust as needed
palette = ['k'] + sns.color_palette("husl", len(binned_data))[:-1]

plt.figure(figsize=(10, 8))


# Plot the data points with error bars
plt.errorbar(
    x=binned_data['Data']['ring_radius_center'],
    y=binned_data['Data']['width_radius_ratio'],
    yerr=binned_data['Data']['ring_width_sem'] / binned_data['Data']['ring_radius_center'],  # Error propagation for ratio
    fmt='o',
    label='Data 2024 year',
    color='k',
    alpha=alpha_reg,
    capsize=4
)
    
    # Fit and plot the regression line
sns.regplot(
    x='ring_radius_center',
    y='width_radius_ratio',
    data=binned_data['Data'],
    scatter=False,  # We already plotted the scatter with error bars
    #label=f'{label} Fit',
    line_kws={'alpha': alpha_reg, 'color': palette[0]},
    ci=95,
    color='k',
    robust=True
)

plt.errorbar(
    x=binned_data['0.015 deg']['ring_radius_center'],
    y=binned_data['0.015 deg']['width_radius_ratio'],
    yerr=binned_data['0.015 deg']['ring_width_sem'] / binned_data['0.015 deg']['ring_radius_center'],  # Error propagation for ratio
    fmt='o',
    label='Sim with mirror_align_random = 0.015 deg',
    color=palette[1],
    alpha=alpha_reg,
    capsize=4
)
    
    # Fit and plot the regression line
sns.regplot(
    x='ring_radius_center',
    y='width_radius_ratio',
    data=binned_data['0.015 deg'],
    scatter=False,  # We already plotted the scatter with error bars
    #label=f'{label} Fit',
    line_kws={'alpha': alpha_reg, 'color': palette[1]},
    ci=95,
    color=palette[1],
    robust=True
)

plt.xlabel('Muon ring radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
#plt.title('Muon Ring Width vs. Ring Radius')
plt.legend(ncol=1, fontsize=18, loc='upper right', frameon=True)

plt.tight_layout()
plt.xlim(1,1.16)
plt.ylim(0.04, 0.07)
plt.grid(alpha=0.4)
plt.tick_params(axis='both', labelsize=15)
#plt.savefig(f'/Users/vdk/muons2024/psf_work/lst_general_meeting/statistica_errors_widthradius_vs_radius_completeness_{COMPLETENESS_CUT}_quantile{QUANTILE_CUT*100}%_radiusrange{min_radius_cut,max_radius_cut}_center{ring_center_distance_cut}_impact{impact_distance_cut}nsb_tune2024.pdf', dpi=200)
plt.show()

# Error metric

In [ ]:
# ratio_099 = binned_data['Data']['width_radius_ratio'] / binned_data['0.015 deg']['width_radius_ratio']
# ratio_error_099 = ratio * np.sqrt(
#     (binned_data['Data']['width_radius_ratio_sem'] / binned_data['Data']['width_radius_ratio'])**2 +
#     (binned_data['0.015 deg']['width_radius_ratio_sem'] / binned_data['0.015 deg']['width_radius_ratio'])**2
# )
# ratio_099

In [ ]:
# ratio_09 = binned_data['Data']['width_radius_ratio'] / binned_data['0.015 deg']['width_radius_ratio']
# ratio_error_09 = ratio * np.sqrt(
#     (binned_data['Data']['width_radius_ratio_sem'] / binned_data['Data']['width_radius_ratio'])**2 +
#     (binned_data['0.015 deg']['width_radius_ratio_sem'] / binned_data['0.015 deg']['width_radius_ratio'])**2
# )
# ratio_09

In [ ]:
# ratio_08 = binned_data['Data']['width_radius_ratio'] / binned_data['0.015 deg']['width_radius_ratio']
# ratio_error_08 = ratio * np.sqrt(
#     (binned_data['Data']['width_radius_ratio_sem'] / binned_data['Data']['width_radius_ratio'])**2 +
#     (binned_data['0.015 deg']['width_radius_ratio_sem'] / binned_data['0.015 deg']['width_radius_ratio'])**2
# )
#ratio_08

In [ ]:
# variable_09 = binned_data['Data']['width_radius_ratio'] / binned_data['0.015 deg']['width_radius_ratio']
# variable_09

In [ ]:
# variable_099 = binned_data['Data']['width_radius_ratio'] / binned_data['0.015 deg']['width_radius_ratio']
# variable_099

In [ ]:
variable_07

In [ ]:
variable_ref = binned_data['Data']['width_radius_ratio'] / binned_data['0.0043 deg']['width_radius_ratio']

plt.figure(figsize=(8,7))
#plt.scatter(binned_data['Data']['ring_radius_center'],variable_ref, c='k',marker='^', label = '')
# plt.scatter(binned_data['Data']['ring_radius_center'],variable_08, c='r',marker='^', label = 'completeness > 0.8', s = 100)
# plt.scatter(binned_data['Data']['ring_radius_center'],variable_09, c='g',marker='^', label = 'completeness > 0.9', s = 100)
# plt.scatter(binned_data['Data']['ring_radius_center'],variable_099, c='b',marker='^', label = 'completeness > 0.99', s = 100)

plt.errorbar(binned_data['Data']['ring_radius_center'], ratio_099, yerr=ratio_error_099, fmt='o',capsize=5, label = 'completeness > 0.99', alpha = 0.7)
plt.errorbar(binned_data['Data']['ring_radius_center'], ratio_09, yerr=ratio_error_09, fmt='o',capsize=5, label = 'completeness > 0.9', alpha = 0.7)
plt.errorbar(binned_data['Data']['ring_radius_center'], ratio_08, yerr=ratio_error_08, fmt='o',capsize=5, label = 'completeness > 0.8', alpha = 0.7)

plt.axhline(1, color='black', linestyle='dotted', lw = 4, alpha = 0.7)
plt.tight_layout()
plt.ylim(0.9,1.15)
plt.xlim(1,1.16)
plt.ylabel('Observ/Sim ring broadening', fontsize=20)
plt.xlabel('Muon ring radius [deg]', fontsize=20)
plt.grid(alpha=0.4)
plt.legend(fontsize=20)
#plt.savefig(f'/Users/vdk/muons2024/psf_work/lst_general_meeting/relative_ring_broad_completeness{ring_completeness_cut}_impact{impact_distance_cut}_radius{min_radius_cut, max_radius_cut}.pdf', dpi=100)

# Relative broadening for all random alignment

In [ ]:
# Define your alpha and palette if not already defined
alpha_reg = 0.7  # Adjust as needed
palette = sns.color_palette('bright', n_colors=len(binned_data))

plt.figure(figsize=(10, 8))

for i, (label, grouped_df) in enumerate(binned_data.items()):
    # Plot the data points with error bars
    plt.errorbar(
        x=grouped_df['ring_radius_center'],
        y=grouped_df['width_radius_ratio'],
        yerr=grouped_df['ring_width_sem'] / grouped_df['ring_radius_center'],  # Error propagation for ratio
        fmt='o',
        label=label,
        color=palette[i],
        alpha=alpha_reg,
        capsize=4
    )
    
    # Fit and plot the regression line
    sns.regplot(
        x='ring_radius_center',
        y='width_radius_ratio',
        data=grouped_df,
        scatter=False,  # We already plotted the scatter with error bars
        #label=f'{label} Fit',
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

plt.xlabel('Muon ring radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
#plt.title('Muon Ring Width vs. Ring Radius')
plt.legend(ncol=1, fontsize=26, loc='upper right', frameon=True)

plt.tight_layout()
plt.xlim(1,1.16)
plt.ylim(0.04, 0.07)
plt.grid(alpha=0.4)
plt.tick_params(axis='both', labelsize=15)
plt.show()

# Additional plots for other df_sim_..._cut dataframes
for df_sim_cut, label in zip(
    [df_sim_00025_cut, df_sim_00043_cut, df_sim_00092_cut, df_sim_002_cut],
    ['0.0025 deg', '0.0043 deg', '0.0092 deg', '0.02 deg']
):
    plt.figure(figsize=(10, 8))
    
    # Plot the data points with error bars
    plt.errorbar(
        x=grouped_df['ring_radius_center'],
        y=grouped_df['width_radius_ratio'],
        yerr=grouped_df['ring_width_sem'] / grouped_df['ring_radius_center'],  # Error propagation for ratio
        fmt='o',
        label=label,
        color=palette[i],
        alpha=alpha_reg,
        capsize=4
    )
    
    # Fit and plot the regression line
    sns.regplot(
        x='ring_radius_center',
        y='width_radius_ratio',
        data=grouped_df,
        scatter=False,  # We already plotted the scatter with error bars
        #label=f'{label} Fit',
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

    plt.xlabel('Muon ring radius [deg]', fontsize=24)
    plt.ylabel('Muon Ring Width / Radius', fontsize=24)
    #plt.title('Muon Ring Width vs. Ring Radius')
    plt.legend(ncol=1, fontsize=26, loc='upper right', frameon=True)

    plt.tight_layout()
    plt.xlim(1,1.16)
    plt.ylim(0.04, 0.07)
    plt.grid(alpha=0.4)
    plt.tick_params(axis='both', labelsize=15)
    plt.show()